In [1]:
# Small Language Model(SLLM) 

# Step 03 - 트랜스포머 허깅페이스 데이터셋(ShareGPT-KO)을 적용하여 Small Language Model(SLLM) 구축 및 텍스트 생성
# - Hugging Face에서 제공하는 FreedomIntelligence/sharegpt-korean 데이터셋을 기반으로 기존 Transformer 언어 모델 구조에 맞춰 구성
# - ShareGPT-KO는 GPT와 사용자 간의 대화 데이터를 한국어로 번역한 고품질 대화형 데이터셋으로, 문장 생성, 대화 모델 학습, 응답 생성 등에 매우 적합함

# 라이브러리 GPU 설정 -> 데이터셋 -> 데이터로더 -> 토크나이저/토큰화 -> SLLM 모델 정의 -> 학습 -> 텍스트 생성 -> 테스트

# 핵심 목적 - 한국어 데이터셋을 사용하여 문장 생성 및 단일문장/대화 흐름을 기반으로 다음 단어 예측
# - 기존 WikiText-2 기반 Transformer 언어 모델 → ShareGPT-KO 대화 데이터셋으로 변경
# - 문장 생성 및 문맥 예측을 위한 시퀀스 학습
# - 단일 문장 또는 대화 흐름을 기반으로 다음 단어 예측

In [35]:
# 1) 라이브러리 GPU 설정
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import re

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Pytorch Version: {torch.__version__}, Device: {device}')

Pytorch Version: 2.2.2, Device: cpu


In [ ]:
# 2) ShareGPT-KO 데이터셋 로드

# Hugging Face에서 ShareGPT-KO 데이터셋 로딩
dataset = load_dataset('FreedomIntelligence/sharegpt-korean', split='train') 

In [7]:
# 3) 데이터셋 내의 대화에서 텍스트 추출
print(dataset)

texts = []
for item in dataset:
    for turn in item['conversations']:
        texts.append(turn['value'])

Dataset({
    features: ['id', 'lang', 'conversations'],
    num_rows: 6014
})


In [39]:
# 4) 토크나이저 모델 로드
# tokenizer_model = 'monologg/kobert'
tokenizer_model = 'beomi/KcBERT-base'  # 안정적인 한국어 토크나이저
auto_tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [40]:
# 5) 토크나이저를 이용하여 토큰화

# 토큰화 함수
def tokenize(text):
    return auto_tokenizer.tokenize(text)

tokens = []
for text in texts: # 텍스트 -> 토큰화
    tokens.extend(tokenize(text))

# 딕셔너리 생성
vocab = sorted(set(tokens))
word2idx = { word:idx for idx, word in enumerate(vocab) } # 단어 -> 인덱스
idx2word = { idx:word for word, idx in word2idx.items() } # 인덱스 -> 단어

Token indices sequence length is longer than the specified maximum sequence length for this model (395 > 300). Running this sequence through the model will result in indexing errors


In [41]:
print(len(vocab))

16538


In [42]:
# 6) 시퀀스 생성 - 입력 시퀀스, 출력 시퀀스 생성
sequence_length = 32
data = []

# 입력 출력 - 32, 32
for i in range(len(tokens) - sequence_length):
    input_seq = tokens[i:i+sequence_length] # 입력 시퀀스
    target_seq = tokens[i+1:i+sequence_length+1] # 출력 시퀀스
    # 입력, 출력 - 단어 -> 인덱스 -> 텐서 변환
    data.append(
        (
            # torch.tensor( [ word2idx[word] for word in input_seq ] ),
            # torch.tensor( [ word2idx[word] for word in target_seq ] )
            torch.tensor( [ word2idx.get(word, word2idx.get('[PAD]', 0)) for word in input_seq ] ),
            torch.tensor( [ word2idx.get(word, word2idx.get('[PAD]', 0)) for word in target_seq ] )
        )
    )

In [43]:
# 7) Dataset 및 DataLoader 생성

# Dataset 설정
class ShareGPTDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        return self.data[index]

# DataLoader 설정
dataloader = DataLoader(ShareGPTDataset(data=data[:50000]), batch_size=8, shuffle=True)

In [44]:
# 8) Transformer 모델 정의
class TransformerSLLM(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, hidden_dim, num_layers, max_len):
        super().__init__()
        # 단어 인덱스를 고정된 차원의 벡터로 변환하는 임베딩 레이어를 정의 vocab_size(단어 사전의 크기), embed_dim(각 단어를 표현할 벡터의 차원, 예시 128차원)
        # 즉 단어 인덱스는 의미를 담고 있지 않음 그래서 임베딩 벡터는 의미적 유사성을 학습, 이 레이어는 vocab_size x embed_dim 크기의 임베딩 행렬을 학습한다
        self.token_embedding = nn.Embedding(vocab_size, embed_dim) # 8086, 128

        # 위지 정보(positional encoding) 입력 시쿼스의 위치 정보를 따로 제공해야 하며, 각 위치에 대해 학습 가능한 위치 벡터를 생성하는 임베딩 레이어
        # max_len(32), embed_dim(128) 위치 정보를 명시적으로 추가해야 문맥을 이해할 수 있음
        self.pos_embedding = nn.Embedding(max_len, embed_dim) # 32, 128

        # d_model(128 입력 및 출력 임베딩 차원), nhead(4 어텐셔 헤드의 수), dim_feedforward(256 내부의 은닉층 크기)
        # 즉 이 레이어는 입력 텐서의 shape이 [seq_len, batch_size, embed_dim]일때 동일한 shape의 출력을 반환하면서 문맥 정보를 강화한다
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            dim_feedforward=hidden_dim,
            batch_first=True
        )

        # Transformer 모델의 인코더 블록 전체를 구성하는 핵심 코드 앞서 정의한 encoder_layer를 여러번 반복해서 깊이 있는 문맥 이해를 가능하게 하는 구조
        # nn.TransformerEncoder 는 하나의 encoder_layer 를 num_layers 만큼 반복하여 전체 인코더 스택을 구성한다
        # Input → EncoderLayer 1 → EncoderLayer 2 → ... → EncoderLayer N → Output
        # 각 레이어는 입력 시퀀시를 받아 문맥 정보를 강화하고, 다음 레이어로 전달한다(반복 횟수가 많을수록 복잡한 문맥과 의미 관계를 더 잘 학습할 수 있음)
        self.transformer = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=num_layers) # encoder_layer, 2

        # 모델의 출력층 정의하는 부분으로 모델이 예측한 임베딩을 단어 사전의 확률 분포로 변환하는 역할, Transformer의 마지막 출력은 [batch_size, seq_len, embed_dim] 형태의 문맥 벡터이다
        # 이 벡터를 vocab_size 차원의 로짓(logits)으로 변환하여, 각 위치에서 어떤 단어가 올지 에측한다, Linear 레이어는 임베딩 공간 -> 단어 공간으로 매핑하는 분류기 역할을 한다
        # embed_dim(Transformer의 출력 차원 예시 128), vocab_size(전체 단어 사전 크기 에시 10000)
        self.fc_out = nn.Linear(embed_dim, vocab_size) # 128, 8086
    
    def forward(self, x):
        # x는 [batch_size, sequence_length] 형태의 텐서, 예를 들어, x.shape = [8, 32]라면:
        # x.size(0) → 배치 크기 (8)
        # x.size(1) → 시퀀스 길이 (32)
        # 즉, seq_len = 32가 된다
        seq_len = x.size(1)

        # seq_len = 32이면 [0, 1, 2, ..., 31] 형태의 위치 인덱스 벡터를 생성
        # 텐서의 shape을 [1, seq_len]으로 변환 → 이렇게 하면 배치 차원과 맞춰서 broadcasting이 가능
        position = torch.arange(0, seq_len, device=x.device).unsqueeze(0)

        # 단어 임베딩과 위치 임베딩을 더해서 최종 입력 벡터를 만들고, 이 합산은 Transformer가 단어 의미 + 위치 정보를 동시에 인식할 수 있게 해준다
        x = self.token_embedding(x) + self.pos_embedding(position)

        # Transformer 인코더에 입력을 전달하여 문맥 정보를 강화
        x = self.transformer(x)

        # Transformer의 출력은 [batch_size, seq_len, embed_dim] 형태의 문맥 벡터이며, fc_out 레이어를 통해 각 위치에서 다음에 올 단어의 확률 분포를 예측
        return self.fc_out(x)

In [34]:
# 9) 학습
model = TransformerSLLM(
    vocab_size=len(vocab),
    embed_dim=128,
    num_heads=4,
    hidden_dim=256,
    num_layers=2,
    max_len=sequence_length
).to(device)

criterion = nn.CrossEntropyLoss() # 손실함수 갑
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 옵티마이저 Adam

for epoch in range(10):
    total_loss = 0
    for inputs, targets in dataloader:
        inputs = inputs.to(device) # 입력 값
        targets = targets.to(device) # 정답 값

        outputs = model(inputs) # 모델 예측 값
        loss = criterion(outputs.view(-1, len(vocab)), targets.view(-1)) # 손실함수 계산 값

        # 오차역전파
        optimizer.zero_grad() # 가중치, 바이어스 파리미터 초기화
        loss.backward() # 미분 연산
        optimizer.step() # 미분 연산 후 가중치,바이어스 파라미터 업데이트

        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss:.4f}')

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1, Loss: 20215.6324
Epoch 2, Loss: 17601.9132
Epoch 3, Loss: 17304.4040
Epoch 4, Loss: 17121.7981
Epoch 5, Loss: 16988.9319
Epoch 6, Loss: 16869.8428
Epoch 7, Loss: 16778.3830
Epoch 8, Loss: 16692.0304
Epoch 9, Loss: 16619.1102
Epoch 10, Loss: 16565.8797


In [ ]:
# 10) 텍스트 생성 및 저장

# torch.topk()로 상위 top_k개의 로짓만 남기고 나머지는 제거, 제거된 로짓은 -inf로 설정되어 softmax에서 확률이 0이 된다.
def top_k_filtering(logits, top_k=50):
    values, _ = torch.topk(logits, top_k)
    min_value = values[:, -1].unsqueeze(1)
    return torch.where(logits < min_value, torch.full_like(logits, float('-inf')), logits)

# softmax 후 누적 확률이 top_p를 넘는 토큰을 제거, top_p=0.9이면 누적 확률이 90%를 넘는 이후의 토큰은 선택되지 않음, 더 자연스럽고 다양성 있는 문장 생성에 효과적이다.
def top_p_filtering(logits, top_p=0.9):
    # logits: [batch_size, vocab_size]
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)

    # top_p 초과하는 부분 제거
    sorted_indices_to_remove = cumulative_probs > top_p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0 # 가장 높은 확률은 유지

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[0, indices_to_remove] = float('-inf')
    return logits

def generate_text(model, start_text, word2idx, idx2word, max_length=50, temperature=1.0, top_k=40, top_p=0.85):
    model.eval()
    tokens = auto_tokenizer.tokenize(start_text)
    # input_ids = [ word2idx.get(word, 0) for word in tokens ]
    input_ids = [ word2idx.get(token, word2idx.get('[PAD]', 0)) for token in tokens ]
    input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)

    for _ in range(max_length):
        with torch.no_grad():
            output = model(input_tensor)
            next_token_logits = output[:, -1, :] / temperature
            logits = top_k_filtering(next_token_logits, top_k)
            logits = top_p_filtering(logits, top_p)
            probabilities = torch.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probabilities, num_samples=1).item()
            input_ids.append(next_token_id)
            input_ids = input_ids[-sequence_length:]
            input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)

    generated_words = [idx2word.get(int(idx), "[UNK]") for idx in input_ids]
    return ' '.join(generated_words)

# 모델 저장
torch.save(model.state_dict(), 'sharegpt_korean_transformer.pt')

In [37]:
# 11) 모델 재생성 후 불러오기
model = TransformerSLLM(
    vocab_size=len(vocab),
    embed_dim=128,
    num_heads=4,
    hidden_dim=256,
    num_layers=2,
    max_len=sequence_length
).to(device)

model.load_state_dict(torch.load("sharegpt_korean_transformer.pt"))
model.eval()


/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TransformerSLLM(
  (token_embedding): Embedding(8086, 128)
  (pos_embedding): Embedding(32, 128)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc_out): Linear(in_features=128, out_features=8086, bias=True)
)

In [ ]:
# 12) 테스트
import re

# 출력 전 후처리 (SentencePiece 토큰 제거 등)
def clean_text(text):
    text = text.replace("▁", "")  # SentencePiece 경계 제거
    text = text.replace("[UNK]", "")  # 알 수 없는 토큰 제거
    text = re.sub(r"[^가-힣a-zA-Z0-9.,!? ]", "", text)  # 특수문자 제거
    text = re.sub(r"(.)\s+\1+", r"\1", text)
    return ' '.join(text.split()).strip()

# 시작 문장 설정
# start_text = "서울의 봄날, 벚꽃이 만개한 거리를 걸으며"
start_text = "봄날 서울에서 가장 아름다운 장소는 어디인가요?"
# start_text = "벚꽃이 만개한 거리를 걸으며 느낀 감정을 말해줘."

# 텍스트 생성 실행
generated = generate_text(model, start_text, word2idx, idx2word, max_length=50)

# 결과 출력
print("📢 생성된 문장:")
print(clean_text(generated))

📢 생성된 문장:
봄 날 서울 에서 가장 아름다운 장소 는 어디 인 가 요 ? O p ic k a ch e x i th on 이 접근 할 수 있습니다 . t ra y p en d 방법 은 추가 하는 클 래 스를 사용 하여 P ri c 2 . 이러한 방법 의 합니다 . 이는 블로그 글 , 9
